task https://stepik.org/lesson/701336/step/7?unit=701405

In [5]:
import csv
import requests
from bs4 import BeautifulSoup

#create file
with open('res_parse4.csv', 'w', encoding='utf-8-sig', newline='') as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerow([
        'Наименование', 'Артикул', 'Бренд', 'Модель', 'Наличие', 'Цена', 'Старая цена',  'Ссылка на карточку с товаром'])    


url = 'https://parsinger.ru/html/index1_page_1.html'
site = 'http://parsinger.ru/html/'

def get_response(url):
    response = requests.get(url=url)
    response.encoding = 'utf-8'  
    return response

def cook_pagination(response,site):
    soup = BeautifulSoup(response.text, 'lxml')
    pages = [f"{site}{link['href']}" for link in soup.find('div', class_='pagen').find_all('a')]        
    return pages

def list_categories(response,site):
    soup = BeautifulSoup(response.text, 'lxml')
    categories = [f"{site}{link['href']}" for link in soup.find('div', class_='nav_menu').find_all('a')]    
    return categories
def flatten(list_of_lists):
    if len(list_of_lists) == 0:
        return list_of_lists
    if isinstance(list_of_lists[0], list):
        return flatten(list_of_lists[0]) + flatten(list_of_lists[1:])
    return list_of_lists[:1] + flatten(list_of_lists[1:])

def cook_products(url, response, site):
    soup = BeautifulSoup(response.text, 'lxml')
    name = soup.find("p", id='p_header' ).text
    article = soup.find('p', class_='article').text.split()[1]
    description = [x.text.split('\n') for x in soup.find('ul', id ='description')]
    description = flatten(description)
    description = [value for value in description if value]
    description = description[:2]

    availability = soup.find("span", id='in_stock').text    
    price = soup.find('span', id='price').text
    old_price = soup.find('span', id='old_price').text   
    
    flat = name, article, *[x.split(':')[1].strip() for x in description if x], availability.split()[2], price, old_price, url
    return flat

for category in list_categories(get_response(url), site):
    #uri = category
    for i in cook_pagination(get_response(category),site):    
        response = get_response(i)
        soup = BeautifulSoup(response.text, 'lxml')
        url_products = [f"{site}{link['href']}" for link in soup.find_all('a', class_='name_item')]
        for url_product in url_products:
            product_info = cook_products(url_product, get_response(url_product), site)        
            with open('res_parse4.csv', 'a', encoding='utf-8-sig', newline='') as file:
                writer = csv.writer(file, delimiter=';')
                writer.writerow(product_info)
print('Файл res_parse4.csv создан')       
    



Файл res_parse4.csv создан
